## Read GHCN precipitation data

In [ ]:
import numpy as np
import pandas as pd

#### Read station inventory

In [ ]:
ghcn_stations = pd.read_table("../input_data/GHCN-Daily/ghcnd-inventory.txt", sep='\s+',
                              names=['ID', 'LATITUDE', 'LONGITUDE', 'ELEMENT', 
                                'FIRST_YEAR', 'LAST_YEAR']) 

In [ ]:
# select for precip stations
ghcn_stations = ghcn_stations.loc[ghcn_stations.ELEMENT == "PRCP"]

# select for midwest region
ghcn_stations = ghcn_stations.loc[(ghcn_stations.LATITUDE >= 37) & 
                        (ghcn_stations.LATITUDE <= 48) & 
                        (ghcn_stations.LONGITUDE >= -104) & 
                        (ghcn_stations.LONGITUDE <= -86)]

# select for full time period
ghcn_stations = ghcn_stations.loc[(ghcn_stations.FIRST_YEAR <= 1981) & 
                       (ghcn_stations.LAST_YEAR >= 2019)]

#### Read/combine precipitation data for midwest stations

In [ ]:
ghcn_dat = []

for year in range(1981, 2020):
    ## read annual data and subset
    t = pd.read_csv("../input_data/GHCN-Daily/years_csv/"+str(year)+".csv", 
                        names = ['ID', 'DATE', 'ELEMENT', 'DATA_VALUE', 'M_FLAG', 
                        'Q_FLAG', 'S_FLAG', 'OBS_TIME'], 
                        dtype = {'ID': 'string', 'DATE': 'int64', 'ELEMENT': 'string', 
                        'DATA_VALUE': 'float64', 'M_FLAG': 'string', 'Q_FLAG': 'string', 
                        'S_FLAG': 'string'})
    t = t.loc[t.ID.isin(ghcn_stations.ID) & 
                 (t.ELEMENT == "PRCP")]
        
    ghcn_dat.append(t)
    
ghcn_dat = pd.concat(ghcn_dat)

In [ ]:
# remove records that were flagged for quality issue 
ghcn_dat = ghcn_dat[ghcn_dat.Q_FLAG.isna()] 

In [ ]:
## Find stations with complete data 
missing_thr = 10 # max percentage of missing dates

record_count = ghcn_dat.groupby(ghcn_dat.ID).count().reset_index()
complete_stations = record_count.loc[(14244-record_count.DATE)/14244 <= missing_thr/100].ID
    
ghcn_dat = ghcn_dat[ghcn_dat.ID.isin(complete_stations)]
    
print(str(len(complete_stations)) + ' stations with < ' + str(missing_thr) + '% of days missing')

In [ ]:
## save combined data
ghcn_dat = ghcn_dat.drop(columns = ["M_FLAG", "Q_FLAG", "S_FLAG", "OBS_TIME", "ELEMENT"]).join(
        ghcn_stations.drop(columns=["ELEMENT", "FIRST_YEAR", "LAST_YEAR"]).set_index("ID"), 
              on = "ID", how = "left")
    
ghcn_dat.to_csv("../processed_data/ghcn_midwest_data.csv", index = False)